In [ ]:
import h5py
import numpy as np
from scipy.interpolate import interp1d

# Plotting
import matplotlib.pyplot as plt

# Profile models
import profiles

# Hydrangea cluster class
import cluster

# MCMC sampler
import emcee
from multiprocessing import Pool

# List of indices of valid Hydrangea clusters
cluster_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 21, 22, 24, 25, 28, 29]

In [ ]:
def normal(x, c, s):
    return -((x-c)**2.)/2./(s**2.)

def lnlike(params, x, y, errors):
    model = profiles.rho(x, params)
    return -0.5*((y-model)**2./errors**2.).sum()

def lnpost(params, x, y, errors, logalphaprior):
    # Posterior for fit_rho below
    rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e = params
    params = np.array([rho_s, r_s, logalpha, r_t, logbeta, loggamma, rho_0, s_e])
    if( (0.<rho_s<1e10) and (0.<rho_0 < 1e10) and (0.1 <= r_s <= 10.)and (0.1 <= r_t <= 20.) and (0.1<s_e<2.)):
        return lnlike(params, x, y, errors) + \
                normal(logalpha, np.log10(0.20), logalphaprior) +\
                normal(logbeta, np.log10(6.), 0.2) +\
                normal(loggamma, np.log10(4.), 0.2) +\
                normal(r_t, 4., 2.)
    return -np.inf

pool = Pool(2)

def fit_rho(x, y, errors, p0=np.array([ 1e-2,  0.5,  -0.3,  1.,  0.6,  0.8, 1e-4,  1.5]), \
            logalphaprior=0.2, \
            nsteps=2500):
    # Fit a 3d profile rho(r) with a DK14 model. 

    nwalkers = 32
    ndim = 8

    pos = [p0 + p0*np.random.randn(ndim)*1e-4 for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnpost, args=(x, y, errors, logalphaprior), pool=pool)
    sampler.run_mcmc(pos, nsteps=nsteps, progress=True)

    lnprob = sampler.get_log_prob()
    params = sampler.get_chain(flat=True)[lnprob.argmax()]

    return sampler.get_chain(flat=True), lnprob[:, :].max(axis=1), params


def get_mass_within(idx, r, stellar=False, zlabel=''):
    # Get mass within a 3d radius
    idx_array = 4
    if(stellar):
        idx_array = 2

    file_data = h5py.File('data/mass_profiles'+zlabel+'.hdf5', 'r')
    x = np.array(file_data['BinLims'])
    y = np.array(file_data['MassProfile3D'][idx, :, idx_array])

    return interp1d(x, y)(r)


def get_profile(idx, type='dm', r_min=0.2, r_max=20., r_min_dm=0.5, nbins=30, zlabel=''):
    # Get a radial profile as a function of 3d radius. 
    
    CEidx = cluster_indices[idx]
    info = np.load('data/sub/hydro'+zlabel+'/'+str(CEidx)+'info.npy')
    
    if(type=='dm'):
        file_data = h5py.File('data/mass_profiles'+zlabel+'.hdf5', 'r')
        x = np.array(file_data['BinLims'])
        y = np.array(file_data['DensityProfile3D'][idx, :, 4])
        
        y = y[(x>r_min_dm) & (x<r_max)]/info[0]/1e10
        x = x[(x>r_min_dm) & (x<r_max)]
        
    else:

        bins = np.geomspace(r_min, r_max, nbins)
        rbin= ((bins[1:]**3. + bins[:-1]**3.) / 2.)**(1./3.)
        volume = 4./3. * np.pi *(bins[1:]**3. - bins[:-1]**3.)        
        
        posc = info[1]
        
        if(type=='sub'):
            mass, pos = np.load('data/sub/hydro'+zlabel+'/'+str(CEidx)+'mass.npy'), \
                            np.load('data/sub/hydro'+zlabel+'/'+str(CEidx)+'pos.npy')
        if(type=='gal'):
            mstar, pos = np.load('data/galaxies'+zlabel+'/'+str(CEidx)+'mstar.npy'), \
                        np.load('data/galaxies'+zlabel+'/'+str(CEidx)+'pos.npy')


        r = np.sqrt( ( (pos-posc)**2.).sum(axis=1))

        if(type=='sub'):
            r = r[(r>r_min) & (r<r_max) & (np.log10(mass)>-1)]
        if(type=='gal'):
            r = r[(r>r_min) & (r<r_max) & (mstar>8.)]

    
        y = np.histogram(r, bins=bins)[0]/volume/len(r)
        x = rbin

    return x, y 

Fit Individual profiles
-----------------------



A parametric mass profile is fit to the subhalo, galaxy and total mass profiles. These fits should be checked for convergence. 

In [ ]:
zlabel = '0.5' # Change fitted redshift. redshift 0 has label '' (empty string)
for idx, CEidx in enumerate(cluster_indices):
    for type in ['dm', 'gal','sub']:
        
        x, y = get_profile(idx, type=type, zlabel=zlabel, r_min=0.2, r_max=15., r_min_dm=0.1, nbins=20)
        
        
        samples, lnprob, bfparams = fit_rho(x, y, y/10., nsteps=500)
        res = (y-profiles.rho(x, bfparams))/y    
            
        '''    
        plt.loglog(x, y)
        plt.loglog(x, profiles.rho(x, bfparams))
        plt.title(type+str(idx))
        plt.show()
        
        plt.plot(lnprob)
        plt.title(type+str(idx))
        plt.show()
        '''
        
        # Save fit results
        dest = "data/fit"+zlabel+"/chains/"+str(CEidx)
        
        np.save(dest+type+"samples", samples)
        np.save(dest+type+"lnprob", lnprob)
        np.save(dest+type+"bfparams", bfparams)
        np.save(dest+type+"res", res)
        
        
        # splashback properties
        dest = "data/fit"+zlabel+"/"+str(CEidx)
        
        rplot = np.geomspace(0.1, 10, 100)
        logy_tot = np.log(profiles.rho(rplot, bfparams))
        derivative = np.gradient(logy_tot, np.log(rplot))

        if(type=='gal'):
            r_sp = rplot[np.argmin(derivative)] 
            M_sp = get_mass_within(idx, r_sp, zlabel=zlabel)
            M_sp_stellar = get_mass_within(idx, r_sp, stellar=True, zlabel=zlabel)
            np.save(dest+type, [r_sp, M_sp, M_sp_stellar])
        else:
            r_sp = rplot[np.argmin(derivative)] 
            M_sp = get_mass_within(idx, r_sp, zlabel=zlabel)
            np.save(dest+type, [r_sp, M_sp])


        plt.semilogx(rplot, derivative)
        plt.axvline(r_sp)
        plt.title(type+str(idx))
        plt.show()

DIRECTIONS
-----------


The preferential direction of a cluster is calculated in one of two ways. Using the subhalo distribution or using the stellar distribution (= orientation of central galaxy). 

These are the "filaments" and "stars" definitions used in the paper. The third mentioned below, "void", is perpendicular to the first. 

In [ ]:
for zlab in ['', '0.5', '1']:
    print(zlab)
    
    for idx, CEidx in enumerate(cluster_indices):
        H = cluster.cluster(idx, zlabel=zlab)
        r200m = H.R200m
        
        print(idx)
        
        for dir_idx in [0, 1, 2]:
            for type in ['filaments', 'stars', 'voids']:
                    
                    
                if(type=='stars'):
                    x1, x2, m = H.get_2d_stars(0, 0.01, dir_idx=dir_idx)
                    size_vec=0.005

                    x1m, x2m = np.mean(x1), np.mean(x2)
                    R2 = x1**2. + x2**2.
                    I_00 = (m*((x2-x2m)**2.)/R2).sum()
                    I_10 = (m*(x1-x1m)*(x2-x2m)/R2).sum()
                    I_11 = (m*((x1-x1m)**2.)/R2).sum()

                    vals, vecs = np.linalg.eig(np.array([[I_00, I_10], [I_10, I_11]]))

                    vec = vecs[vals.argmin()]
                    theta = np.arctan2(vec[0], vec[1])


                if(type=='filaments'):
                    #use filaments
                    x1, x2, m = H.get_2d_subs(r200m, r200m*5, dir_idx=dir_idx)
                    size_vec=5

                    a, b = np.histogram(np.arctan(x1/x2), bins=np.linspace(-np.pi/2, np.pi/2, 20))
                    b = (b[:-1]+b[1:])/2.
                    theta = b[a.argmax()]

                if(type=='voids'):
                    x1, x2, m = H.get_2d_subs(r200m, r200m*5, dir_idx=dir_idx)
                    size_vec=5

                    a, b = np.histogram(np.arctan(x1/x2), bins=np.linspace(-np.pi/2, np.pi/2, 20))
                    b = (b[:-1]+b[1:])/2.
                    theta = b[a.argmin()]


                plt.title(str(CEidx)+' '+type+' '+str(dir_idx))
                plt.scatter(x1, x2, s=0.001)
                plt.plot([0, np.sin(theta)*size_vec], [0, np.cos(theta)*size_vec], c='k')
                plt.show()

                np.save('data/direction'+zlab+'/'+str(CEidx)+str(dir_idx)+type+'.npy', theta)
                
    